In [1]:
using Pkg
Pkg.add("InteractBase")
Pkg.add("CSSUtil")
Pkg.add("WebIO")
Pkg.add("JSExpr")
Pkg.add("Observables")
Pkg.add("AlphaGo")
Pkg.add("Flux")
Pkg.add("BSON")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  99.9 %0.0 %]  14.5 %============>                            ]  29.0 %43.3 %>                ]  57.8 % [=============================>           ]  72.3 %86.7 %>  ]  94.4 % Resolving package versions...
 Installed CodecZlib ──────── v0.5.1
 Installed SpecialFunctions ─ v0.7.2
 Installed DoubleFloats ───── v0.3.4
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [944b1d66] ↑ CodecZlib v0.5.0 ⇒ v0.5.1
  [497a8b3b] ↑ DoubleFloats v0.3.3 ⇒ v0.3.4
  [276daf66] ↑ SpecialFunctions v0.7.1 ⇒ v0.7.2
  Building SpecialFunctions → `~/.julia/packages/SpecialFunctions/fvheQ/deps/build.log`
  Building CodecZlib ───────→ `~/.julia/packages/CodecZlib/DAjXH/deps/build.log`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [

In Lecture 9, we have learned about multi-agent systems and adverserial games. A central algorithm in game playing is **MiniMax**.

MiniMax essentially searches the entire space of moves possible by both players at any given time till the game reaches an end. This is how it maximizes reward while the opponent is trying to minimize it. We saw an optimization called alpha-beta pruning for MiniMax which is a great improvement but only by a factor of 2. This might help in games such as chess (an AI agent can be twice as smart on the same computer now assuming it uses a plausible heuristic).

TicTacToe has $9! = 362880$ possible states, roughly half of them are valid states. This kind of state space can be fully searched by computers.

Some games such as Go are way too complicated.

By one estimate there are:

```
2081681993819799846994786333448627702865224
5388453054842563945682092741961273801537852
5648451698519643907259916015628128546089888
314427129715319317557736620397247064840935
```

States!

That certainly cannot be handled by even the biggest supercomputers or an earth filled with GPUs.

A big search tree. The branching factor in Go is 150-200.

## Infeasible state space

Go is a googol ($10^{100}$) times more complex than chess. It has more states than there are atoms in the universe. It has been a formidable challenge for AI.


In March 2016 DeepMind's AlphaGo Lee became the first program to beat a human expert -- 18-time world champion Lee Sedol. (DeepBlue beat Gary Kasparov in chess in 1997 -- super human ability in Go took 19 years longer).

The key insight of this software was to use a neural network to learn the game by looking at how humans play (using plays from ameteur players on internet Go servers) it to begin with, but later by playing the program against itself.

In 19 Oct 2017 DeepMind unweiled a newer, more elegant version of AlphaGo -- the AlphaGo Zero. This version learnt the game all on its own through playing millions of games with its best self. In 3 days, AlphaGo Zero trained enough to beat the first version (AlphaGo Lee) 100 games to 0!

## AlphaGo Zero's algorithm

### Background

**Q-learning** with neural networks:

- **Goal-orientation:** The agent's goal is to figure out what is the best move at any given state.
- Recall the $Q$ function we used for value iteration (it's the expected future reward). $Q : S \times A \rightarrow \mathbb{R}$
- Usually we represent $Q$ as a table: we actually store a value for every action at every action. But as you can tell in the case of Go, $Q$ is going to be too big!
- **One idea is to approximate $Q$ with a function that requires less storage.** A function such as a neural network.
  - Start off with a random neural network
  - Keep playing the game to improve the $Q$ function it approximates
  
This is not exactly what AlphaGo zero uses, but it can be used to play a lot of games including Breakout. AlphaGo Zero marries Q-learning with Monte-carlo tree search.

**Monte carlo tree search (MCTS)**


![](https://nikcheerla.github.io/deeplearningschool//media/branching.jpg)

One way programs can search the state space graph is to **pick a random sample of branches**, playing each one many times to see which ones have a better probability of winning down the tree. This is called Monte-carlo tree search.

**Approximating a function that guides MCTS**

AlphaGo zero creates a neural network $f_{\theta}$ which approximates the probability distribution among all branches for the next move. Then the sampling algorithm draws from this probability distribution. It also uses the same neural network to guess the expected probability of winning from the state it is applied to.


$$(\textbf{p}, v) = f_{\theta}(s)$$

Here $\textbf{p}$ maps every action available at state $s$ to a probability value that it should be sampled.



**Self-play**

A neural network needs to be told what are good moves and what are not in order to train itself to be better at the game. AlphaGo Zero keeps improving by playing against itself. Winning the game gets a reward of 1, losing
This is the simple algorithm that runs all of AlphaGo Zero!

In [2]:
using WebIO, InteractBase, JSExpr, CSSUtil, Observables

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e8f953a4abf5e4095b83e225fc5e11254819fa44-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e8f953a4abf5e4095b83e225fc5e11254819fa44-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [3]:
const btn_style = Dict("width"=> 36px, "height"=>36px, "borderRadius" => 4px, "margin"=>2px)
function board()
    scope = Scope()
    click = scope["clicks"] = Observable{Any}(nothing)
    scope["notif"] = Observable("")
    for i=1:3, j=1:3
        scope["cell-$i-$j"] = Observable(" ")
    end
    btns = [button(label=scope["cell-$i-$j"], style=btn_style,
        events=Dict("click" => @js () -> $click[] = $((i,j))))
              for i=1:3, j=1:3]
    grid = hbox(mapslices((x...)->vbox(x...), reshape(btns, (3,3)), dims=1)...)
    scope.dom = hbox(grid, hskip(1em), scope["notif"])(alignitems("center"))
    scope
end

function makechan(ob)
    c = Channel{Any}(0)
    on(ob) do x
        @async put!(c, x)
    end
    c
end

makechan (generic function with 1 method)

In [4]:
b = board()

Scope("scope-144fb72c-68df-4043-b1dd-66935bd7f233", Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-02b2d8fb-3f9c-4098-8c0a-aa8fa9e8c244", Node{DOM}(DOM(:html, :button), Any[Observable{String}("ob_03", " ", Any[])], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:events=>Dict("click"=>JSString("(function (){return WebIO.setval({\"name\":\"clicks\",\"scope\":\"scope-144fb72c-68df-4043-b1dd-66935bd7f233\",\"id\":\"ob_01\",\"type\":\"observable\"},[1,1])})")),:style=>Dict("height"=>36.0px,"borderRadius"=>4.0px,"width"=>36.0px,"margin"=>2.0px)), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clicks"=>(Observable{Int64}("ob_12", 0, Any[SyncCallback(Scope(#= circular reference @-6 =#), SyncCallback(Scope(#= circular reference @-7 =#), ##40#42{Scope,String}(Scope(#= circular reference @-8 =#), "clicks")))]), nothing)), Set(String[]), nothing, Any["knockout"=>"/home/shashi/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/home/shashi/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/home/shashi/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"clicks\\\":0}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"clicks\"].subscribe((function (val){!(this.valueFromJulia[\"clicks\"]) ? (WebIO.setval({\"name\":\"clicks\",\"scope\":\"knockout-component-02b2d8fb-3f9c-4098-8c0a-aa8fa9e8c244\",\"id\":\"ob_12\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"clicks\"]=false}),self)]\n        \n    }\n    self.model = new AppViewModel();\n    self.valueFromJulia = {};\n    for (var key in json_data) {\n        self.valueFromJulia[key] = false;\n    }\n    ko.applyBindings(self.model, self.dom);\n}\n")]),"clicks"=>Any[JSString("(function (val){return (val!=this.model[\"clicks\"]()) ? (this.valueFromJulia[\"clicks\"]=true, this.model[\"clicks\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:1), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)))], Dict{Symbol,Any}(:className=>"field"), 1), Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-888c9008-a650-4be9-bba4-f7bb77e321ae", Node{DOM}(DOM(:html, :button), Any[Observable{String}("ob_06", " ", Any[])], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:events=>Dict("click"=>JSString("(function (){return WebIO.setval({\"name\":\"clicks\"

In [5]:
using AlphaGo, Flux
using BSON: @load

function load_model(str, n, env::AlphaGo.GameEnv)
  @load str*"/agz_base-$n.bson" bn
  @load str*"/agz_value-$n.bson" value
  @load str*"/agz_policy-$n.bson" policy

  @load str*"/weights/agz_base-$n.bson" bn_weights
  @load str*"/weights/agz_value-$n.bson" val_weights
  @load str*"/weights/agz_policy-$n.bson" pol_weights

  Flux.loadparams!(bn, bn_weights)
  Flux.loadparams!(value, val_weights)
  Flux.loadparams!(policy, pol_weights)

  NeuralNet(env; base_net=bn, value=value, policy=policy)
end



┌ Info: Recompiling stale cache file /home/shashi/.julia/compiled/v1.0/AlphaGo/hIRw1.ji for AlphaGo [da2c498e-f8db-5b64-9cc4-315cb361752b]
└ @ Base loading.jl:1187


load_model (generic function with 1 method)

In [6]:
import AlphaGo: GameEnv, MCTSPlayer, initialize_game!, N, tree_search!, is_done

function play(env::AlphaGo.GameEnv, nn, human_moves, accepted_moves,
              notif; tower_height = 6, num_readouts = 800, mode = 0
  ) #mode=0, human starts with black, else starts with white

  @assert 0 ≤ tower_height ≤ 19

  az = MCTSPlayer(env, nn, num_readouts = num_readouts, two_player_mode = true)

  initialize_game!(az)
  num_moves = 0

  mode = mode == 0 ? mode : 1
  while !is_done(az)
    if num_moves % 2 == mode
      notif[] = "Your turn"
      mv = take!(human_moves)
      move = Tuple(mv)
    else
      notif[] = "AlphaZero's turn"
      current_readouts = N(az.root)
      readouts = az.num_readouts

      while N(az.root) < current_readouts + readouts
        tree_search!(az)
      end

      move = pick_move(az)
      #println(to_kgs(move, az.env))
    end
    if play_move!(az, move)
      accepted_moves[] = (move, num_moves % 2)
      num_moves += 1
    end
  end

  #println(az.root.position)

  winner = result(az.root.position)
  set_result!(az, winner, false)
  mode = mode == 0 ? -1 : 1
  notif[] = string("Result ", az.result_string)
end


play (generic function with 1 method)

In [7]:
# Play with a specific neural network
function play_with(gametype, nn::NeuralNet)
    b = board()

    moves = Observable{Any}(nothing)
    on(moves) do (mv, mode,)
        i, j = mv
        b["cell-$i-$j"][] = string(mode)
    end

    t= @async play(gametype, nn, makechan(b["clicks"]), moves, b["notif"])

    b,t
end


# play with the dumbest network first:
gametype = AlphaGo.GomokuEnv(3,3) # the specific case which is tictactoe
nn = load_model("models", "1", gametype) # there are 50 levels we have trained, second argument is the level

b,t = play_with(gametype, nn)
b

Scope("scope-65e4b9f0-eceb-42e7-b01a-dc350cd5b4a0", Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-f4c026b0-8d1a-41eb-b6ff-2dc8d0d683d5", Node{DOM}(DOM(:html, :button), Any[Observable{String}("ob_33", " ", Any[])], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:events=>Dict("click"=>JSString("(function (){return WebIO.setval({\"name\":\"clicks\",\"scope\":\"scope-65e4b9f0-eceb-42e7-b01a-dc350cd5b4a0\",\"id\":\"ob_31\",\"type\":\"observable\"},[1,1])})")),:style=>Dict("height"=>36.0px,"borderRadius"=>4.0px,"width"=>36.0px,"margin"=>2.0px)), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clicks"=>(Observable{Int64}("ob_42", 0, Any[SyncCallback(Scope(#= circular reference @-6 =#), SyncCallback(Scope(#= circular reference @-7 =#), ##40#42{Scope,String}(Scope(#= circular reference @-8 =#), "clicks")))]), nothing)), Set(String[]), nothing, Any["knockout"=>"/home/shashi/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/home/shashi/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/home/shashi/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"clicks\\\":0}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"clicks\"].subscribe((function (val){!(this.valueFromJulia[\"clicks\"]) ? (WebIO.setval({\"name\":\"clicks\",\"scope\":\"knockout-component-f4c026b0-8d1a-41eb-b6ff-2dc8d0d683d5\",\"id\":\"ob_42\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"clicks\"]=false}),self)]\n        \n    }\n    self.model = new AppViewModel();\n    self.valueFromJulia = {};\n    for (var key in json_data) {\n        self.valueFromJulia[key] = false;\n    }\n    ko.applyBindings(self.model, self.dom);\n}\n")]),"clicks"=>Any[JSString("(function (val){return (val!=this.model[\"clicks\"]()) ? (this.valueFromJulia[\"clicks\"]=true, this.model[\"clicks\"](val)) : undefined})")]), ConnectionPool(Channel{Any}(sz_max:9223372036854775807,sz_curr:1), Set(AbstractConnection[]), Channel{AbstractConnection}(sz_max:32,sz_curr:0)))], Dict{Symbol,Any}(:className=>"field"), 1), Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-95e9ae18-f4b4-4ade-92fc-0ad0424fac07", Node{DOM}(DOM(:html, :button), Any[Observable{String}("ob_36", " ", Any[])], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:events=>Dict("click"=>JSString("(function (){return WebIO.setval({\"name\":\"clicks\"

In [8]:
@manipulate for level = slider(1:50, label="Experience level -- (1 level=200 games): ", value=1)
    gametype = AlphaGo.GomokuEnv(3,3) # the specific case which is tictactoe
    nn = load_model("models", string(level), gametype) # there are 50 levels we have trained, second argument is the level

    play_with(gametype, nn)[1]
end
    

Widget{:manipulate,Any}(DataStructures.OrderedDict{Symbol,Any}(:level=>Widget{:slider,Int64}(DataStructures.OrderedDict{Symbol,Any}(:changes=>Observable{Int64}("ob_63", 0, Any[SyncCallback(Scope("knockout-component-9adfe736-6dd2-43ab-9f1e-d5d5adfa8232", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["Experience level -- (1 level=200 games): "], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"text-align:right;width:18%")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>50,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"flex-grow:1; margin: 0 2%")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: value")), 0)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"width:18%")), 1)], Dict{Symbol,Any}(:attributes=>Dict("style"=>"display:flex; justify-content:center; align-items:center;")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64}(#= circular reference @-6 =#), nothing),"value"=>(Observable{Int64}("ob_62", 1, Any[SyncCallback(Scope(#= circular reference @-6 =#), SyncCallback(Scope(#= circular reference @-7 =#), ##40#42{Scope,String}(Scope(#= circular reference @-8 =#), "value"))), #g#15{##16#17{##18#20,Observable{Any}},Tuple{Observable{Int64}}}(##16#17{##18#20,Observable{Any}}(##18#20(), Observable{Any}("ob_86", Scope("scope-183a03aa-5df8-458f-89e1-ed8dd1e4e79c", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-f064c37a-e59e-4b38-b0a1-290047b7532c", Node{DOM}(DOM(:html, :button), Any[Observable{String}("ob_66", " ", Any[])], Dict{Symbol,Any}(:attributes=>Dict{Any,Any}("data-bind"=>"click : function () {this.clicks(this.clicks()+1)}"),:className=>"button is-primary",:events=>Dict("click"=>JSString("(function (){return WebIO.setval({\"name\":\"clicks\",\"scope\":\"scope-183a03aa-5df8-458f-89e1-ed8dd1e4e79c\",\"id\":\"ob_64\",\"type\":\"observable\"},[1,1])})")),:style=>Dict("height"=>36.0px,"borderRadius"=>4.0px,"width"=>36.0px,"margin"=>2.0px)), 1), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("clicks"=>(Observable{Int64}("ob_75", 0, Any[SyncCallback(Scope(#= circular reference @-6 =#), SyncCallback(Scope(#= circular reference @-7 =#), ##40#42{Scope,String}(Scope(#= circular reference @-8 =#), "clicks")))]), nothing)), Set(String[]), nothing, Any["knockout"=>"/home/shashi/.julia/packages/Knockout/JIqpG/src/../assets/knockout.js", "knockout_punches"=>"/home/shashi/.julia/packages/Knockout/JIqpG/src/../assets/knockout_punches.js", "/home/shashi/.julia/packages/InteractBase/Q4IkI/src/../assets/style.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0

In [9]:
import AlphaGo: GameEnv, MCTSPlayer, initialize_game!, N, tree_search!, is_done

function play(env::AlphaGo.GameEnv, nn, human_moves, accepted_moves,
              notif; tower_height = 6, num_readouts = 800)

  @assert 0 ≤ tower_height ≤ 19

  az = MCTSPlayer(env, nn, num_readouts = num_readouts, two_player_mode = true)

  initialize_game!(az)
  num_moves = 0

  mode = 0
  while !is_done(az)
    if num_moves % 2 == mode
      notif[] = "Your turn"
      mv = take!(human_moves)
      move = Tuple(mv)
    else
      notif[] = "AlphaZero's turn"
      current_readouts = N(az.root)
      readouts = az.num_readouts

      while N(az.root) < current_readouts + readouts
        tree_search!(az)
      end

      move = pick_move(az)
      #println(to_kgs(move, az.env))
    end
    if play_move!(az, move)
      accepted_moves[] = (move, num_moves % 2)
      num_moves += 1
    end
  end

  #println(az.root.position)

  winner = result(az.root.position)
  set_result!(az, winner, false)
  mode = mode == 0 ? -1 : 1
  notif[] = string("Result ", az.result_string)
end


play (generic function with 1 method)

## References

- **Mastering the game of Go without human knowledge**  -- Nature (Oct. 2017)
David Silver, Julian Schrittwieser, Karen Simonyan, Ioannis Antonoglou, Aja Huang, Arthur Guez, Thomas Hubert, Lucas Baker, Matthew Lai, Adrian Bolton, Yutian Chen, Timothy Lillicrap, Fan Hui, Laurent Sifre, George van den Driessche, Thore Graepel & Demis Hassabis

- **AlphaGo Zero Explained - On AI** blog post https://nikcheerla.github.io/deeplearningschool/2018/01/01/AlphaZero-Explained/

- **AlphaGo.jl** - Julia implementation of the AlphaGo Zero problem on Go and Gomoku -- a generalization of tic-tac-toe

In [10]:
include("optimal.jl")

minimize (generic function with 1 method)

In [11]:
@time maximize(@SArray([-1 0 0; 0 1 0; 0 0 0]), 1)

  3.779345 seconds (5.32 M allocations: 270.213 MiB, 8.87% gc time)


(0, 7420, (2, 1))

In [12]:
factorial(9)

362880

In [91]:
import AlphaGo: GameEnv, MCTSPlayer, initialize_game!, N, tree_search!, is_done
adapt_state(x) = State(x.board)

function play_optimal(env::AlphaGo.GameEnv, nn; tower_height = 6, num_readouts = 800, mode=0)
  @assert 0 ≤ tower_height ≤ 19

  az = MCTSPlayer(env, nn, num_readouts = num_readouts, two_player_mode = true)

    states = []
  initialize_game!(az)
  num_moves = 0

  while !is_done(az)
        #println(az.root.position)
        push!(states,  copy(az.root.position.board))
    if num_moves % 2 == mode
        if num_moves == 0
            move = (rand(1:3), rand(1:3))
        else
            mv = maximize(adapt_state(az.root.position), 1)
            move = mv[3]
        end
    else
      current_readouts = N(az.root)
      readouts = az.num_readouts

      while N(az.root) < current_readouts + readouts
        tree_search!(az)
      end

      move = pick_move(az)
      #println(to_kgs(move, az.env))
    end
    if play_move!(az, move)
      num_moves += 1
    end
  end

  #println(az.root.position)
    winner(adapt_state(az.root.position))
end


play_optimal (generic function with 1 method)

In [98]:
    gametype = AlphaGo.GomokuEnv(3,3) # the specific case which is tictactoe
    nn = load_model("models", "50", gametype) # there are 50 levels we have trained, second argument is the level

    plays = [play_optimal(gametype, nn) for i=1:100]

100-element Array{Int8,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [99]:
sum(plays.==0)

100

In [ ]:
stats = [begin
    gametype = AlphaGo.GomokuEnv(3,3) # the specific case which is tictactoe
    nn = load_model("models", string(i), gametype) # there are 50 levels we have trained, second argument is the level

    plays = [play_optimal(gametype, nn) for i=1:100]
        end for i=1:50]

In [44]:
using ProgressMeter

┌ Info: Recompiling stale cache file /home/shashi/.julia/compiled/v1.0/ProgressMeter/3V8n6.ji for ProgressMeter [92933f4c-e287-5a05-a399-4b506db050ca]
└ @ Base loading.jl:1187


In [45]:
]add ProgressMeter

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [88]:
sum(last.(plays).==1)

11

In [84]:
filter(x->x[2] .== 1, plays)[1][1][2]

3×3 Array{Int8,2}:
 1  0  0
 0  0  0
 0  0  0